<a href="https://colab.research.google.com/github/byiringiroscar/static_file_chatbot/blob/main/RssbChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Fetch the content from the front page
content = requests.get('https://www.rssb.rw/').content

In [ ]:
#Preview of the Content
content

In [3]:
#Initiating the BS4 parser
page = BeautifulSoup(content, 'html.parser')

In [4]:
#Get all the links in the website
links = page.find_all('a')

In [5]:
len(links)

35

In [6]:
# main links fetching and main titles
list_of_links = [] #list()
list_of_titles = []

for item in links: #loop over the list of links
  if not str(item['href']) == '/':
    if len(item.text) > 0:
      if str(item['href']).startswith('http'):
        lin = str(item['href'])
        list_of_links.append(lin)
        list_of_titles.append(item.text)
        
      else:
        lin = str(item['href'])
        domain = 'https://www.rssb.rw' + lin
        list_of_links.append(domain)
        list_of_titles.append(item.text)
    
    

  
  

In [ ]:
list_of_links

In [7]:

len(list_of_titles)

28

In [8]:
# take every link then go inside then get all links associated with it
def get_individual_link(fina_l): 
  try:
    content_individual_link = requests.get(fina_l).content # get content for each link
    page_individual_link = BeautifulSoup(content_individual_link, 'html.parser', from_encoding="iso-8859-1")
    final_link_individual = page_individual_link.findAll('a')
    for links_ind in final_link_individual:
      if not str(links_ind['href']) == '/':
        if len(links_ind.text) > 0:
          if str(links_ind['href']).startswith('http'):
            lin = str(links_ind['href'])
            list_of_links.append(lin)
            list_of_titles.append(links_ind.text)
          else:
            lin = str(links_ind['href'])
            domain = 'https://www.rssb.rw' + lin
            list_of_links.append(domain)
            list_of_titles.append(links_ind.text)
  except:
    pass



In [ ]:
len(list_of_titles)

In [9]:
for i in range(len(list_of_links)):
  fina_links = list_of_links[i]
  fin_li_official = get_individual_link(fina_links)

In [ ]:
len(list_of_links)

In [142]:
import re
real_link = []
for lin in list_of_links:
  if bool(re.search('rssb', lin)):
    if not lin.endswith('docx') and not lin.endswith('pdf') and not lin.endswith('xl') and not lin.endswith('xlsx') and not lin.endswith('html') and not lin.endswith('gov') and not lin.endswith('doc') and not lin.endswith('zip') and not lin.endswith('mdb') and not lin.endswith('xls'):
      real_link.append(lin)





In [143]:
len(real_link)

420

In [144]:
# remove duplicate in order to see all rssb what content inside then append it in list called list_of_links
real_link = list(set(real_link))
len(real_link)

75

In [166]:
def get_content_link_rssb_links(link_rssb):
  content_rssb = requests.get(link_rssb).content
  page_rssb = BeautifulSoup(content_rssb, 'html.parser')
  links_rssb = page_rssb.find('p')
  if links_rssb:
    list_of_titles.append(links_rssb.text)
    list_of_links.append(link_rssb)


In [169]:
for lin_rssb in real_link:
  try:
    get_content_link_rssb_links(lin_rssb)
  except:
    pass


In [174]:

len(list_of_titles)
len(list_of_links)

706

In [175]:
df = pd.DataFrame()
df['title'] = list_of_titles
df['link'] = list_of_links
df['description'] = None

In [176]:
df.describe()

,title,link,description
count,706,706,0
unique,239,191,0
top,schemes,https://www.imisanzu.rssb.rw/,NaN
freq,20,23,NaN


In [177]:
#Drop all the row with empty title columns
df = df.dropna(subset=['title'])

In [181]:
df.describe()

,title,link,description
count,239,239,0
unique,239,187,0
top,schemes,https://www.rssb.rw/rssb-invests-in-katapult-a...,NaN
freq,1,5,NaN


In [183]:
#Check whehter the link is full formated link
is_full_link = df.link.str.startswith('http')

#Check whether the link is pdf documents
is_pdf = df.link.str.contains('.pdf')

In [184]:
df['is_full_link'] = is_full_link #Create a new col
df['is_pdf'] = is_pdf #create a new col

In [ ]:
df

In [186]:
# data cleaning in dataframe
import re 

def preprocessing(titles : list):
  cleaned_titles = []
  for title in titles:
    title = title.strip()
    title = title.lower()
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    title = url_pattern.sub(r'', title)
    title = html_pattern.sub(r'', title)
    title = re.sub(r"[^\w\d'\s]+", ' ', title)
    cleaned_titles.append(title)
  return cleaned_titles


df['title'] = preprocessing(df['title'])

# Start Building the chatbot

## Building the intent file

In [192]:
question = """Guide me to {}?
Where do you get {}?
What is the link to the {}?
How do you get {}?
Who is in Charge of {}?
how can I get {}?
which way to get {}?
I need to get {}?
direct me to {}
please how can I {}?
{}"""

answers = """Here is the link {}.
You can use this link {} for more information.
thanks for contacting use this {} for more detail.
ok try this {} for more information.
take a look on this {}.
check out on this link {} for more detail."""

In [ ]:
answers.split('\n')

In [ ]:
!git clone https://github.com/patrickloeber/pytorch-chatbot.git

In [195]:
import json
with open('/content/pytorch-chatbot/intents.json', 'r') as intent_file:
  intents__ = json.load(intent_file)

In [196]:
greet_intents = []
for intet in intents__['intents']:
  greet_intents.append(intet)


In [197]:
intents = []

for index in range(len(df)):
  intent = {}
  tag = df.iloc[index]['title']
  intent['tag'] = tag
  intent['patterns'] = list()
  intent['responses'] = list()
  # intent['responses'] = [f'You can use this link {df.iloc[index].link} for more information']
  for i in range(len(question.split('\n'))):
    intent['patterns'].append(question.split('\n')[i].format(intent['tag']))
  for i in range(len(answers.split('\n'))):
    intent['responses'].append(answers.split('\n')[i].format(df.iloc[index].link))

  intents.append(intent)

In [198]:
intents.extend(greet_intents)

In [ ]:
file = {'intents': intents}
file

In [200]:
import json

with open('/content/pytorch-chatbot/intents.json', 'w+') as intent_file:
  intent_file.seek(0)
  intent_file.truncate()
  json.dump(file, intent_file)

In [201]:
%cd pytorch-chatbot/

/content/pytorch-chatbot


In [202]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!python train.py

In [204]:
import random
import json

import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Oscar"
print("Let's chat! (type 'quit' to exit)")

def query(question : str):
  # sentence = "do you use credit cards?"
  sentence = question

  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device)

  output = model(X)
  _, predicted = torch.max(output, dim=1)

  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]
  if prob.item() > 0.75:
      for intent in intents['intents']:
          if tag == intent["tag"]:
              return f"{bot_name}: {random.choice(intent['responses'])}"
  else:
      return f"{bot_name}: I do not understand..."

Let's chat! (type 'quit' to exit)


In [205]:
response = query('how are you ?')

In [207]:
response

'Oscar: Hi there, how can I help?'

In [ ]:
!pip install langdetect

In [209]:

import requests
import json


class translator:
    api_url = "https://translate.googleapis.com/translate_a/single"
    client = "?client=gtx&dt=t"
    dt = "&dt=t"

    #fROM English to Kinyarwanda
    def translate(text : str , target_lang : str, source_lang : str):
        sl = f"&sl={source_lang}"
        tl = f"&tl={target_lang}"
        r = requests.get(translator.api_url+ translator.client + translator.dt + sl + tl + "&q=" + text)
        return json.loads(r.text)[0][0][0]

In [210]:
resp = translator.translate(text='Mpereza amakuru yerekeye Belise', target_lang='fr', source_lang='rw')

In [211]:
resp

'Je recherche des informations sur Belize'

In [212]:
from langdetect import detect

def process_question(text : str):
  if text.lower() == 'hi' or text.lower() == 'hey' or text.lower() == 'how are you' or text.lower() == 'hello':
    resp = text.lower()
    source_lang = 'en'
    return resp, source_lang
  else:
    source_lang = detect(text)
    resp = translator.translate(text=text, target_lang='en', source_lang=source_lang)
    return resp, source_lang

def process_answer(text : str, source_lang):
  resp = translator.translate(text=text, target_lang=source_lang, source_lang='en')
  return resp

In [216]:
def process(QUESTION: str):
  USER_QUERY, SL = process_question(QUESTION) #Translate the original question into english and store the source lang
  RESPONSE = query(USER_QUERY) #Asking th chatbot question
  ORIGINAL_RESPONSE = process_answer(RESPONSE, SL)
  return ORIGINAL_RESPONSE

process("The mission of RSSB is to manage and promote social security in Rwanda")

'Oscar: ok try this https://www.rssb.rw/fileadmin/user_upload/rssb_anti_fraud_policy_april_2018pdf.pdf for more information.'

#FAST API

In [ ]:
!pip install fastapi pyngrok uvicorn nest-asyncio

In [ ]:
!ngrok authtoken 2HJTD4eU4nX4kqE0VD1YVDlysnq_4pvNxMFU5pe3Q4ERkSeHP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from IPython.core.interactiveshell import asyncio
import uvicorn
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
import schemas
app = FastAPI()

templates = Jinja2Templates(directory='/content/templates')
app.mount("/content/static", StaticFiles(directory="/content/static"), name="static")


# storage = FastAPI(title='FASTAPI')
# fakeDatabase = {
#     1: {'task': "hello"},
# }

@app.get('/', response_class=HTMLResponse)
def home(request: Request):
    return templates.TemplateResponse("base.html", {"request": request})
@app.post("/predict")
async def predict(item: schemas.Item):
    response = process(item.message)
    message = {"answer": response}
    return message



# @storage.get('/')
# async def index():
#   return {"data": "successfully"}
# @storage.get("/{lang}")
# async def getItem(lang: str):
#   if lang:
#     sol = process(lang)
#     new_id = len(fakeDatabase.keys()) + 1
#     fakeDatabase[new_id] = {'task': sol}
#     return fakeDatabase
#   else:
#     return {"data": "failed"}

  

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print("REST API started")
print("Your public API URL:", ngrok_tunnel.public_url)
print("You can for example open the following URL in your browser: {}?target_lang=en&text=Hallo%20Welt".format(ngrok_tunnel.public_url))

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [70]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


REST API started
Your public API URL: http://e198-35-189-174-205.ngrok.io
You can for example open the following URL in your browser: http://e198-35-189-174-205.ngrok.io?target_lang=en&text=Hallo%20Welt
INFO:     41.216.107.113:0 - "GET / HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "GET /style.css HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     41.216.107.113:0 - "POST /predict HTTP/1.1" 200 OK
I